# DATA PREPROCESSING

In this notebook, we will reorganize, clean up the dataset and create fingerprints for model training. My personal preference is to keep all libraries to be used at the beginning of the notebook and import them at the same time. Let's start with navigating to the project's root directory.

In [1]:
%cd ..

/Users/mk/Projects/contrastive-learning


In [2]:
import rdkit
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from rdkit.Chem import MolFromSmiles, DataStructs, MolStandardize
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
from rdkit import RDLogger
from chembl_structure_pipeline import standardize_mol
from sklearn.model_selection import train_test_split
from hydra import initialize, compose

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


[21:11:43] Initializing Normalizer


Disable RDKit warning messages before we start.

In [3]:
logger = RDLogger.logger()
logger.setLevel(RDLogger.ERROR)

Load parameters file (aka config file).

In [4]:
with initialize(config_path="../", version_base=None):
    cfg = compose(config_name="params.yaml")

## Data Cleaning

Load the data and take a peek into it.

In [5]:
path = cfg.preprocessing.fpath
data = pd.read_csv(path)
data.head()

/var/folders/py/ptqbyrdn7yv9tj8d9ry9gpnw0000gn/T/ipykernel_71735/3202889972.py:2: DtypeWarning: Columns (129,134,141,169,174,181,209,214,221,249,254,261,289,294,301,329,334,341,369,374,381,409,414,421,449,454,461,489,494,501,529,534,541,569,574,581,609,614,621,649,654,661,689,694,701,729,734,741,769,774,781,809,814,821,849,854,861,889,894,901,929,934,941,969,974,981,1009,1014,1021,1049,1054,1061,1089,1094,1101,1129,1134,1141,1169,1174,1181,1209,1214,1221,1249,1254,1261,1289,1294,1301,1329,1334,1341,1369,1374,1381,1409,1414,1421,1449,1454,1461,1489,1494,1501,1529,1534,1541,1569,1574,1581,1609,1614,1621,1649,1654,1661,1689,1694,1701,1729,1734,1741,1769,1774,1781,1809,1814,1821,1849,1854,1861,1889,1894,1901,1929,1934,1941,1969,1974,1981,2009,2014,2021) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)


,PUBCHEM_RESULT_TAG,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_EXT_DATASOURCE_SMILES,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,PUBCHEM_ACTIVITY_URL,PUBCHEM_ASSAYDATA_COMMENT,Phenotype-Replicate_1,Potency-Replicate_1,...,Activity at 23.64 uM-Replicate_51,Activity at 52.95 uM-Replicate_51,Activity at 115.2 uM-Replicate_51,Activity at 299.6 uM-Replicate_51,Activity at 1087.9 uM-Replicate_51,Activity at 2306.0 uM-Replicate_51,Activity at 5157.0 uM-Replicate_51,Activity at 11530.0 uM-Replicate_51,Activity at 25780.0 uM-Replicate_51,Activity at 57660.0 uM-Replicate_51
0,1,144205501,17931.0,CN1C(=S)CN=C(C2=C1C=CC(=C2)Cl)C3=CC=CC=C3,Inconclusive,21,http://assay.nih.gov/htsws/rest/display/tox21-...,NaN,Inhibitor,15.4871,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,144206117,3045407.0,CO[C@H]1[C@@H](C[C@@H]2CN3CCC4=C([C@H]3C[C@@H]...,Active,85,http://assay.nih.gov/htsws/rest/display/tox21-...,NaN,Inhibitor,2.1876,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,144206325,14708.0,C[C@]12CC[C@H]3[C@H]([C@@H]1CC[C@]2(C)O)CC[C@@...,Active,61,http://assay.nih.gov/htsws/rest/display/tox21-...,NaN,Inhibitor,17.3768,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,144206326,3085168.0,CC1=NC=C(C(=N1)N)CN(C=O)/C(=C(\CCO)/SS/C(=C(/C...,Active,41,http://assay.nih.gov/htsws/rest/display/tox21-...,NaN,Inhibitor,15.4871,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,144206328,54158.0,CC1=CC(=C(C(=C1NC(=O)CN(CC(=O)O)CC(=O)O)C)Br)C,Inactive,0,http://assay.nih.gov/htsws/rest/display/tox21-...,NaN,Inactive,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's check data types, NaNs, etc.

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9667 entries, 0 to 9666
Columns: 2049 entries, PUBCHEM_RESULT_TAG to Activity at 57660.0 uM-Replicate_51
dtypes: float64(1838), int64(54), object(157)
memory usage: 151.1+ MB


There are more than 2000 columns in the dataset. Since I intent to make a quick example, I'll train a binary classification model and will only need the PUBCHEM_ACTIVITY_OUTCOME column to train my model on and PUBCHEM_EXT_DATASOURCE_SMILES column to create inputs to the model. Therefore, I'll create a new dataframe with the first 5 columns of the original dataframe. I'll keep the first three columns in case I need some identifiers after I clean the molecules and canonicalize the SMILES.

In [7]:
data_cropped = data.iloc[:,0:5]
data_cropped

,PUBCHEM_RESULT_TAG,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_EXT_DATASOURCE_SMILES,PUBCHEM_ACTIVITY_OUTCOME
0,1,144205501,17931.0,CN1C(=S)CN=C(C2=C1C=CC(=C2)Cl)C3=CC=CC=C3,Inconclusive
1,2,144206117,3045407.0,CO[C@H]1[C@@H](C[C@@H]2CN3CCC4=C([C@H]3C[C@@H]...,Active
2,3,144206325,14708.0,C[C@]12CC[C@H]3[C@H]([C@@H]1CC[C@]2(C)O)CC[C@@...,Active
3,4,144206326,3085168.0,CC1=NC=C(C(=N1)N)CN(C=O)/C(=C(\CCO)/SS/C(=C(/C...,Active
4,5,144206328,54158.0,CC1=CC(=C(C(=C1NC(=O)CN(CC(=O)O)CC(=O)O)C)Br)C,Inactive
...,...,...,...,...,...
9662,9663,251919987,23664984.0,CCC1=C(C(=CC=C1)CC)N(COC)C(=O)CS(=O)(=O)[O-].[...,Inactive
9663,9664,251919988,2724429.0,CCCC(C(CO)O)O,Inactive
9664,9665,251919989,13059052.0,CC(C)(C1=CC(=C(C=C1)O)C2=CC=CC=C2)C3=CC(=C(C=C...,Active
9665,9666,251919990,14178379.0,C1=CC=C(C=C1)COC2=CC=C(C=C2)S(=O)(=O)C3=CC=CC=C3O,Inactive


Let's check the datatypes again.

In [8]:
data_cropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9667 entries, 0 to 9666
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   PUBCHEM_RESULT_TAG             9667 non-null   int64  
 1   PUBCHEM_SID                    9667 non-null   int64  
 2   PUBCHEM_CID                    9524 non-null   float64
 3   PUBCHEM_EXT_DATASOURCE_SMILES  9524 non-null   object 
 4   PUBCHEM_ACTIVITY_OUTCOME       9667 non-null   object 
dtypes: float64(1), int64(2), object(2)
memory usage: 377.7+ KB


The dataframe is indexed properly. Column #3,4 have missing values. PUBCHEM_CID is an identifier column, but there are two more identifier columns. So, col #3 is okay, but col #4 is our input column. So, we need to drop NaNs in the PUBCHEM_EXT_DATASOURCE_SMILES column.

In [9]:
data_cropped.dropna(subset=["PUBCHEM_EXT_DATASOURCE_SMILES"], axis=0,inplace=True)
data_cropped

,PUBCHEM_RESULT_TAG,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_EXT_DATASOURCE_SMILES,PUBCHEM_ACTIVITY_OUTCOME
0,1,144205501,17931.0,CN1C(=S)CN=C(C2=C1C=CC(=C2)Cl)C3=CC=CC=C3,Inconclusive
1,2,144206117,3045407.0,CO[C@H]1[C@@H](C[C@@H]2CN3CCC4=C([C@H]3C[C@@H]...,Active
2,3,144206325,14708.0,C[C@]12CC[C@H]3[C@H]([C@@H]1CC[C@]2(C)O)CC[C@@...,Active
3,4,144206326,3085168.0,CC1=NC=C(C(=N1)N)CN(C=O)/C(=C(\CCO)/SS/C(=C(/C...,Active
4,5,144206328,54158.0,CC1=CC(=C(C(=C1NC(=O)CN(CC(=O)O)CC(=O)O)C)Br)C,Inactive
...,...,...,...,...,...
9661,9662,251919986,47364.0,C[As+](C)(C)CC(=O)[O-],Inactive
9662,9663,251919987,23664984.0,CCC1=C(C(=CC=C1)CC)N(COC)C(=O)CS(=O)(=O)[O-].[...,Inactive
9663,9664,251919988,2724429.0,CCCC(C(CO)O)O,Inactive
9664,9665,251919989,13059052.0,CC(C)(C1=CC(=C(C=C1)O)C2=CC=CC=C2)C3=CC(=C(C=C...,Active


In [10]:
data_cropped['PUBCHEM_ACTIVITY_OUTCOME'].value_counts()

PUBCHEM_ACTIVITY_OUTCOME
Inactive        6771
Active          1779
Inconclusive     974
Name: count, dtype: int64

There are active, inactive and inconclusive compounds. I'll drop inconclusives to binarize the model outcome and simplify the problem for this example.

In [11]:
inconclusives = [id for id, row in data_cropped.iterrows() if row["PUBCHEM_ACTIVITY_OUTCOME"] == "Inconclusive"]
data_cropped.drop(labels=inconclusives, axis=0, inplace=True)
data_cropped.reset_index(drop=True, inplace=True)
data_cropped

,PUBCHEM_RESULT_TAG,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_EXT_DATASOURCE_SMILES,PUBCHEM_ACTIVITY_OUTCOME
0,2,144206117,3045407.0,CO[C@H]1[C@@H](C[C@@H]2CN3CCC4=C([C@H]3C[C@@H]...,Active
1,3,144206325,14708.0,C[C@]12CC[C@H]3[C@H]([C@@H]1CC[C@]2(C)O)CC[C@@...,Active
2,4,144206326,3085168.0,CC1=NC=C(C(=N1)N)CN(C=O)/C(=C(\CCO)/SS/C(=C(/C...,Active
3,5,144206328,54158.0,CC1=CC(=C(C(=C1NC(=O)CN(CC(=O)O)CC(=O)O)C)Br)C,Inactive
4,6,144206329,92140.0,CCCC(=O)OC[C@H]([C@H]([C@H](CN1C2=C(C=C(C(=C2)...,Active
...,...,...,...,...,...
8545,9662,251919986,47364.0,C[As+](C)(C)CC(=O)[O-],Inactive
8546,9663,251919987,23664984.0,CCC1=C(C(=CC=C1)CC)N(COC)C(=O)CS(=O)(=O)[O-].[...,Inactive
8547,9664,251919988,2724429.0,CCCC(C(CO)O)O,Inactive
8548,9665,251919989,13059052.0,CC(C)(C1=CC(=C(C=C1)O)C2=CC=CC=C2)C3=CC(=C(C=C...,Active


I will not do anymore cleaning on this dataset. It is pretty much ready for fingerprint creation. So, I'll save it as the intermediate dataset.

In [12]:
data_cropped.to_csv(cfg.preprocessing.savepath, index=False)

## Preparation of Training and Validation Sets

In this part of the notebook, we will create fingerprints from SMILES first. Then, we will split the dataset into training and validation sets for model training. Let's standardize compounds and create fingerprints. First, get mols from SMILES.

In [13]:
ROMols= [MolFromSmiles(smiles) for smiles in tqdm(data_cropped['PUBCHEM_EXT_DATASOURCE_SMILES'])]

  0%|          | 0/8550 [00:00<?, ?it/s]

[21:11:47] Explicit valence for atom # 1 Si, 8, is greater than permitted


I checked mols and found some bad molecules (due to the input). Therefore, I will use a try-except loop to catch bad mols and drop them from the dataframe.

In [14]:
std_mols = []
bad_mols = []
bad_ids = []
desalter = MolStandardize.fragment.LargestFragmentChooser()


In [15]:
for id,mol in enumerate(tqdm(ROMols)):
    try:
        std_mol = standardize_mol(mol)
        desalt_mol = desalter.choose(std_mol)
        std_mol = standardize_mol(desalt_mol)
        std_mols.append(std_mol)
    except:
        bad_mols.append(mol)
        bad_ids.append(id)

  0%|          | 0/8550 [00:00<?, ?it/s]

Let's check the bad molecules.

In [16]:
print(bad_mols)
print(bad_ids)

[None]
[4937]


There is nothing left of SMILES 4937 after the cleanup. I'll drop this mol from ROMols and, also, from the dataframe.

In [17]:
ROMols.pop(bad_ids[0])
data_cropped.drop(labels=bad_ids, axis=0, inplace=True)
data_cropped.reset_index(drop=True, inplace=True)

Now, we can create fingerprints and split the dataset into training and validation sets.

In [18]:
def create_ecfp(
        mol: rdkit.Chem.rdchem.Mol,
        radius: int = 3,
        nbits: int = 1024
        ) -> np.ndarray:
    
    mfbitvector = GetMorganFingerprintAsBitVect(mol, radius, nbits)
    arr = np.zeros((1,0))
    DataStructs.ConvertToNumpyArray(mfbitvector, arr)
    
    return arr


In [19]:
ecfps = [create_ecfp(mol, cfg.preprocessing.radius, cfg.preprocessing.nbits) for mol in tqdm(std_mols)]

  0%|          | 0/8549 [00:00<?, ?it/s]

Next, binarize PUBCHEM_ACTIVITY_OUTCOMEs.

In [20]:
labels = [int(1) if activity == "Active" else int(0) for activity in data_cropped["PUBCHEM_ACTIVITY_OUTCOME"]]

Finally, split the dataset into training and validation sets. I'll use 80:20 stratified split.

In [21]:
X_train, X_val, y_train, y_val = train_test_split(ecfps, labels, test_size=0.2, random_state=7)

In [22]:
print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")

Training set size: 6839
Validation set size: 1710


Now, save training and validation sets. I'll use the parquet file format to preserve the file formats.

In [23]:
df_train = pd.DataFrame({"ecfp": X_train, "y": y_train})
df_val = pd.DataFrame({"ecfp":X_val, "y": y_val})
print(df_train)
print(df_val)
df_train.to_parquet(cfg.preprocessing.trainpath, index=False)
df_val.to_parquet(cfg.preprocessing.valpath, index=False)

                                                   ecfp  y
0     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  0
1     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  0
2     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  0
3     [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  0
4     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  0
...                                                 ... ..
6834  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  0
6835  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  0
6836  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  1
6837  [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  0
6838  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  0

[6839 rows x 2 columns]
                                                   ecfp  y
0     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  0
1     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  1
2     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  0
3     [0.0, 0.0, 0.0, 0.0, 0.0,

/Users/mk/opt/anaconda3/envs/deeplearning/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
